<a href="https://colab.research.google.com/github/benalaka/SpeechComprehension/blob/main/topic_speech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import sys
import glob
import csv
import requests
from time import sleep
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')
path_audio="/content/drive/MyDrive/Log_PhD/wav"
path_prompts="/content/drive/MyDrive/Log_PhD/prompts"
new_path_audio="/content/drive/MyDrive/Log_PhD/wav_dataset"

API_KEY = "b5f7a87aa3ae453093cb09057f800cc2"

AUDIO_FILE = '/content/drive/MyDrive/Log_PhD/wav_dataset'
UPLOAD_ENDPOINT = 'https://api.assemblyai.com/v2/upload'
TRANSCRIPT_ENDPOINT = 'https://api.assemblyai.com/v2/transcript'
OUTPUT_TRANSCRIPT_FILE = '/content/drive/MyDrive/Log_PhD/topic_detection/speech-to-text-tutorial.txt'
CSV_FILE = '/content/drive/MyDrive/Log_PhD/topic_detection/topic.csv'

# adding header to csv file
headerList = ["Text", "Topic_Label", "Relevance"]
with open(CSV_FILE,'w') as f:
  writer = csv.writer(f, dialect='excel')
  writerObj = csv.writer(f)
  writerObj.writerow(headerList)

def read_audio_file(file):
    """Helper method that reads in audio files"""
    with open(file, 'rb') as f:
        while True:
            data = f.read(5242880)
            if not data:
                break
            yield data
#for filename in os.listdir(AUDIO_FILE):
for filename in glob.iglob(f'{AUDIO_FILE}/*'):
  # Create the headers for request
  headers = {
      'authorization': API_KEY, 
      'content-type': 'application/json'
  }



  res_upload = requests.post(
      UPLOAD_ENDPOINT, 
      headers=headers, 
      data=read_audio_file(filename)
  )
  
  upload_url = res_upload.json()['upload_url']


  res_transcript = requests.post(
      TRANSCRIPT_ENDPOINT,
      headers=headers,
      json={
          'audio_url': upload_url,
          'iab_categories': True,
      }, 
  )
  res_transcript_json = res_transcript.json()


  status = ''
  while status != 'completed':    
      res_result = requests.get(
          os.path.join(TRANSCRIPT_ENDPOINT, res_transcript_json['id']),
          headers=headers
      )
      status = res_result.json()['status']
      print(f'Status: {status}')

      if status == 'error':
          sys.exit('Audio file failed to process.')
      elif status != 'completed':
        sleep(10)

      
  with open(OUTPUT_TRANSCRIPT_FILE, 'a') as f:
      f.write(res_result.json()['text']+"\n")
      

      #data frame
      summary_df = pd.DataFrame(columns = ["Text", "Topic_Label", "Relevance"])
      for idx, (key, value) in enumerate(res_result.json()['iab_categories_result']['summary'].items()):
        #First write to csv
        with open(CSV_FILE,'a') as f:
          writer = csv.writer(f, dialect='excel')
          writer.writerow([(res_result.json()['text']), key, round(value, 2)])
          #then add to data frame
          summary_df.loc[summary_df.shape[0]] = [(res_result.json()['text']), key, round(value, 2)]
       

  
      
  print(f'Transcript file saved under {OUTPUT_TRANSCRIPT_FILE}')
  print(res_result.json()['iab_categories_result']['summary'])
summary_df.head()



